In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
#offertimes.to_csv('offertimes.csv')
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
#inc_prod_num.to_csv('inc_prod_num.csv')
choice_prob = raw_jd_choice['clicknum'] / raw_jd_choice.groupby('clickset')['clicknum'].transform('sum')
raw_jd_choice['choice prob'] = choice_prob

assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

# all purchase options in each assortment
purchaseset = raw_jd_choice['purchaseset']
purchaseset_list = []
for pset in purchaseset:
    num_lst = json.loads(pset)
    purchaseset_list.append(num_lst)
raw_jd_choice['purchaseset'] = purchaseset_list

purchasenum_list = raw_jd_choice['clicknum'].tolist()
jd_choice=pd.DataFrame({'clickset':clickset_list,'purchaseset':purchaseset_list,'purchasenum':purchasenum_list,'choice prob':choice_prob})

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
collection = []
for cset in clickset_list:
    if cset not in collection:
        collection.append(cset)
print('there are {} different assortments'.format(len(collection)))

assortment_info_df['assortments'] = collection
choice_collection = np.zeros((n,len(collection)))
global_num = 0
for i in range(len(collection)):
    local_num =0
    while(local_num < inc_prod_num[i]):
        if purchaseset_list[global_num][0] in collection[i]:
            choice_collection[purchaseset_list[global_num][0]][i] = choice_prob[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print('error!')
full_choice_collection = pd.DataFrame(choice_collection)
#full_choice_collection.to_csv('instances/full_instances/full_choice.csv')


there are 9 different products
there are 134 different assortments


In [4]:
assortment_info_df_20 = assortment_info_df[(assortment_info_df['offer_times']>=20)]
assortments_20 = []
for i in range(len(assortment_info_df_20['assortments'])):
    assortments_20.append(assortment_info_df_20['assortments'][i])


jd_choice_20 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_20:
        jd_choice_20.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_20 = assortment_info_df_20['assortments']
choice_collection_20 = np.zeros((n,len(collection_20)))

frequency_collection_20 = np.zeros((n,len(collection_20)))

purchaseset_list_20 = jd_choice_20['purchaseset'].tolist()

inc_prod_num_20 = assortment_info_df_20['includ_prod_num']
choice_prob_20 = jd_choice_20['choice prob']
frequency_20 = jd_choice_20['purchasenum']

global_num = 0
for i in range(len(collection_20)):
    local_num =0
    while(local_num < inc_prod_num_20[i]):
        if purchaseset_list_20[global_num][0] in assortments_20[i]:
            choice_collection_20[purchaseset_list_20[global_num][0]][i] = choice_prob_20[global_num]
            frequency_collection_20[purchaseset_list_20[global_num][0]][i] = frequency_20[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_20[global_num])
            print(assortments_20[i])
            print('error!')
            break
#print(global_num)  

choice_check_20 = pd.DataFrame(choice_collection_20)
#choice_check_20.to_csv('instances/full_instances/full_20.csv')
frequency_collection_20 = frequency_collection_20.astype(int)
frequency_check_20 = pd.DataFrame(frequency_collection_20)
#frequency_check_20.to_csv('instances/full_instances/frequency_20.csv')

offer_times_20 = assortment_info_df_20['offer_times'].tolist()
print('the number of assortments with at least 20 purchases is',len(assortments_20))

the number of assortments with at least 20 purchases is 29


In [5]:
assortment_info_df_30 = assortment_info_df[(assortment_info_df['offer_times']>=30)]
assortments_30 = []
for i in range(len(assortment_info_df_30['assortments'])):
    assortments_30.append(assortment_info_df_30['assortments'][i])


jd_choice_30 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_30:
        jd_choice_30.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_30 = assortment_info_df_30['assortments']
choice_collection_30 = np.zeros((n,len(collection_30)))

frequency_collection_30 = np.zeros((n,len(collection_30)))

purchaseset_list_30 = jd_choice_30['purchaseset'].tolist()

inc_prod_num_30 = assortment_info_df_30['includ_prod_num']
choice_prob_30 = jd_choice_30['choice prob']
frequency_30 = jd_choice_30['purchasenum']

global_num = 0
for i in range(len(collection_30)):
    local_num =0
    while(local_num < inc_prod_num_30[i]):
        if purchaseset_list_30[global_num][0] in assortments_30[i]:
            choice_collection_30[purchaseset_list_30[global_num][0]][i] = choice_prob_30[global_num]
            frequency_collection_30[purchaseset_list_30[global_num][0]][i] = frequency_30[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_30[global_num])
            print(assortments_30[i])
            print('error!')
            break
#print(global_num)  

choice_check_30 = pd.DataFrame(choice_collection_30)
#choice_check_30.to_csv('instances/full_instances/full_30.csv')
frequency_collection_30 = frequency_collection_30.astype(int)
frequency_check_30 = pd.DataFrame(frequency_collection_30)
#frequency_check_30.to_csv('instances/full_instances/frequency_30.csv')

offer_times_30 = assortment_info_df_30['offer_times'].tolist()
print('the number of assortments with at least 30 purchases is',len(assortments_30))

the number of assortments with at least 30 purchases is 24


In [6]:
assortment_info_df_40 = assortment_info_df[(assortment_info_df['offer_times']>=40)]
assortments_40 = []
for i in range(len(assortment_info_df_40['assortments'])):
    assortments_40.append(assortment_info_df_40['assortments'][i])


jd_choice_40 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_40:
        jd_choice_40.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_40 = assortment_info_df_40['assortments']
choice_collection_40 = np.zeros((n,len(collection_40)))

frequency_collection_40 = np.zeros((n,len(collection_40)))

purchaseset_list_40 = jd_choice_40['purchaseset'].tolist()

inc_prod_num_40 = assortment_info_df_40['includ_prod_num']
choice_prob_40 = jd_choice_40['choice prob']
frequency_40 = jd_choice_40['purchasenum']

global_num = 0
for i in range(len(collection_40)):
    local_num =0
    while(local_num < inc_prod_num_40[i]):
        if purchaseset_list_40[global_num][0] in assortments_40[i]:
            choice_collection_40[purchaseset_list_40[global_num][0]][i] = choice_prob_40[global_num]
            frequency_collection_40[purchaseset_list_40[global_num][0]][i] = frequency_40[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_40[global_num])
            print(assortments_40[i])
            print('error!')
            break
#print(global_num)  

choice_check_40 = pd.DataFrame(choice_collection_40)
#choice_check_40.to_csv('instances/full_instances/full_40.csv')
frequency_collection_40 = frequency_collection_40.astype(int)
frequency_check_40 = pd.DataFrame(frequency_collection_40)
#frequency_check_40.to_csv('instances/full_instances/frequency_40.csv')

offer_times_40 = assortment_info_df_40['offer_times'].tolist()
print('the number of assortments with at least 40 purchases is',len(assortments_40))

the number of assortments with at least 40 purchases is 19


In [7]:
assortment_info_df_50 = assortment_info_df[(assortment_info_df['offer_times']>=50)]
assortments_50 = []
for i in range(len(assortment_info_df_50['assortments'])):
    assortments_50.append(assortment_info_df_50['assortments'][i])


jd_choice_50 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_50:
        jd_choice_50.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_50 = assortment_info_df_50['assortments']
choice_collection_50 = np.zeros((n,len(collection_50)))

frequency_collection_50 = np.zeros((n,len(collection_50)))

purchaseset_list_50 = jd_choice_50['purchaseset'].tolist()

inc_prod_num_50 = assortment_info_df_50['includ_prod_num']
choice_prob_50 = jd_choice_50['choice prob']
frequency_50 = jd_choice_50['purchasenum']

global_num = 0
for i in range(len(collection_50)):
    local_num =0
    while(local_num < inc_prod_num_50[i]):
        if purchaseset_list_50[global_num][0] in assortments_50[i]:
            choice_collection_50[purchaseset_list_50[global_num][0]][i] = choice_prob_50[global_num]
            frequency_collection_50[purchaseset_list_50[global_num][0]][i] = frequency_50[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_50[global_num])
            print(assortments_50[i])
            print('error!')
            break
#print(global_num)  

choice_check_50 = pd.DataFrame(choice_collection_50)
#choice_check_50.to_csv('instances/full_instances/full_50.csv')
frequency_collection_50 = frequency_collection_50.astype(int)
frequency_check_50 = pd.DataFrame(frequency_collection_50)
#frequency_check_50.to_csv('instances/full_instances/frequency_50.csv')

offer_times_50 = assortment_info_df_50['offer_times'].tolist()
print('the number of assortments with at least 50 purchases is',len(assortments_50))

the number of assortments with at least 50 purchases is 15


In [8]:
assortment_info_df_60 = assortment_info_df[(assortment_info_df['offer_times']>=60)]
assortments_60 = []
for i in range(len(assortment_info_df_60['assortments'])):
    assortments_60.append(assortment_info_df_60['assortments'][i])


jd_choice_60 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_60:
        jd_choice_60.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_60 = assortment_info_df_60['assortments']
choice_collection_60 = np.zeros((n,len(collection_60)))

frequency_collection_60 = np.zeros((n,len(collection_60)))

purchaseset_list_60 = jd_choice_60['purchaseset'].tolist()

inc_prod_num_60 = assortment_info_df_60['includ_prod_num']
choice_prob_60 = jd_choice_60['choice prob']
frequency_60 = jd_choice_60['purchasenum']

global_num = 0
for i in range(len(collection_60)):
    local_num =0
    while(local_num < inc_prod_num_60[i]):
        if purchaseset_list_60[global_num][0] in assortments_60[i]:
            choice_collection_60[purchaseset_list_60[global_num][0]][i] = choice_prob_60[global_num]
            frequency_collection_60[purchaseset_list_60[global_num][0]][i] = frequency_60[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_60[global_num])
            print(assortments_60[i])
            print('error!')
            break
#print(global_num)  

choice_check_60 = pd.DataFrame(choice_collection_60)
#choice_check_60.to_csv('instances/full_instances/full_60.csv')
frequency_collection_60 = frequency_collection_60.astype(int)
frequency_check_60 = pd.DataFrame(frequency_collection_60)
#frequency_check_60.to_csv('instances/full_instances/frequency_60.csv')

offer_times_60 = assortment_info_df_60['offer_times'].tolist()
print('the number of assortments with at least 60 purchases is',len(assortments_60))

the number of assortments with at least 60 purchases is 13


In [9]:
assortment_info_df_75 = assortment_info_df[(assortment_info_df['offer_times']>=75)]
assortments_75 = []
for i in range(len(assortment_info_df_75['assortments'])):
    assortments_75.append(assortment_info_df_75['assortments'][i])


jd_choice_75 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_75:
        jd_choice_75.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_75 = assortment_info_df_75['assortments']
choice_collection_75 = np.zeros((n,len(collection_75)))

frequency_collection_75 = np.zeros((n,len(collection_75)))

purchaseset_list_75 = jd_choice_75['purchaseset'].tolist()

inc_prod_num_75 = assortment_info_df_75['includ_prod_num']
choice_prob_75 = jd_choice_75['choice prob']
frequency_75 = jd_choice_75['purchasenum']

global_num = 0
for i in range(len(collection_75)):
    local_num =0
    while(local_num < inc_prod_num_75[i]):
        if purchaseset_list_75[global_num][0] in assortments_75[i]:
            choice_collection_75[purchaseset_list_75[global_num][0]][i] = choice_prob_75[global_num]
            frequency_collection_75[purchaseset_list_75[global_num][0]][i] = frequency_75[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_75[global_num])
            print(assortments_75[i])
            print('error!')
            break
#print(global_num)  

choice_check_75 = pd.DataFrame(choice_collection_75)
#choice_check_75.to_csv('instances/full_instances/full_75.csv')
frequency_collection_75 = frequency_collection_75.astype(int)
frequency_check_75 = pd.DataFrame(frequency_collection_75)
#frequency_check_75.to_csv('instances/full_instances/frequency_75.csv')

offer_times_75 = assortment_info_df_75['offer_times'].tolist()
print('the number of assortments with at least 75 purchases is',len(assortments_75))

the number of assortments with at least 75 purchases is 12


In [10]:
assortment_info_df_100 = assortment_info_df[(assortment_info_df['offer_times']>=100)]
assortments_100 = []
for i in range(len(assortment_info_df_100['assortments'])):
    assortments_100.append(assortment_info_df_100['assortments'][i])

jd_choice_100 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_100:
        jd_choice_100.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_100 = assortment_info_df_100['assortments']
choice_collection_100 = np.zeros((n,len(collection_100)))

frequency_collection_100 = np.zeros((n,len(collection_100)))

purchaseset_list_100 = jd_choice_100['purchaseset'].tolist()

inc_prod_num_100 = assortment_info_df_100['includ_prod_num']
choice_prob_100 = jd_choice_100['choice prob']
frequency_100 = jd_choice_100['purchasenum']

global_num=0
for i in range(len(collection_100)):
    local_num =0
    while(local_num < inc_prod_num_100[i]):
        if purchaseset_list_100[global_num][0] in assortments_100[i]:
            choice_collection_100[purchaseset_list_100[global_num][0]][i] = choice_prob_100[global_num]
            frequency_collection_100[purchaseset_list_100[global_num][0]][i] = frequency_100[global_num]
            global_num = global_num +1
            local_num = local_num +1

choice_check_100 = pd.DataFrame(choice_collection_100)
#choice_check_100.to_csv('instances/full_instances/full_100.csv')
frequency_collection_100 = frequency_collection_100.astype(int)
frequency_check_100 = pd.DataFrame(frequency_collection_100)
#frequency_check_100.to_csv('instances/full_instances/frequency_100.csv')

offer_times_100 = assortment_info_df_100['offer_times'].tolist()
print('the number of assortments with at least 100 purchases is',len(assortments_100))

the number of assortments with at least 100 purchases is 11


In [11]:
assortment_info_df_15 = assortment_info_df[(assortment_info_df['offer_times']>=15)]
assortments_15 = []
for i in range(len(assortment_info_df_15['assortments'])):
    assortments_15.append(assortment_info_df_15['assortments'][i])


jd_choice_15 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_15:
        jd_choice_15.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_15 = assortment_info_df_15['assortments']
choice_collection_15 = np.zeros((n,len(collection_15)))

frequency_collection_15 = np.zeros((n,len(collection_15)))

purchaseset_list_15 = jd_choice_15['purchaseset'].tolist()

inc_prod_num_15 = assortment_info_df_15['includ_prod_num']
choice_prob_15 = jd_choice_15['choice prob']
frequency_15 = jd_choice_15['purchasenum']

global_num = 0
for i in range(len(collection_15)):
    local_num =0
    while(local_num < inc_prod_num_15[i]):
        if purchaseset_list_15[global_num][0] in assortments_15[i]:
            choice_collection_15[purchaseset_list_15[global_num][0]][i] = choice_prob_15[global_num]
            frequency_collection_15[purchaseset_list_15[global_num][0]][i] = frequency_15[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_15[global_num])
            print(assortments_15[i])
            print('error!')
            break
#print(global_num)  

choice_check_15 = pd.DataFrame(choice_collection_15)
#choice_check_15.to_csv('instances/full_instances/full_15.csv')
frequency_collection_15 = frequency_collection_15.astype(int)
frequency_check_15 = pd.DataFrame(frequency_collection_15)
#frequency_check_15.to_csv('instances/full_instances/frequency_15.csv')

offer_times_15 = assortment_info_df_15['offer_times'].tolist()
print('the number of assortments with at least 15 purchases is',len(assortments_15))

the number of assortments with at least 15 purchases is 38


both MDM and RUM represent the data with approximately 99% confidence interval,  we set both z = 2.5758293 for both RUM and MDM

In [12]:
confidence_level = 0.99
alpha = 1 - confidence_level
# Find z-score for the given confidence level
target_z_score = stats.norm.ppf(1 - alpha / 2)

stardard_error_20 = np.zeros(frequency_collection_20.shape)
for i in range(stardard_error_20.shape[0]):
    for j in range(stardard_error_20.shape[1]):
        if frequency_collection_20[i][j]!=0:
            stardard_error_20[i][j] = np.sqrt((1-choice_collection_20[i][j])/frequency_collection_20[i][j])
            
lb_20 = np.zeros(frequency_collection_20.shape)
ub_20 = np.zeros(frequency_collection_20.shape)
for i in range(lb_20.shape[0]):
    for j in range(lb_20.shape[1]):
        if frequency_collection_20[i][j]!=0:
            lb_20[i][j] = choice_collection_20[i][j] * (1-target_z_score*stardard_error_20[i][j]) 
            ub_20[i][j] = choice_collection_20[i][j] * (1+target_z_score*stardard_error_20[i][j])

stardard_error_30 = np.zeros(frequency_collection_30.shape)
for i in range(stardard_error_30.shape[0]):
    for j in range(stardard_error_30.shape[1]):
        if frequency_collection_30[i][j]!=0:
            stardard_error_30[i][j] = np.sqrt((1-choice_collection_30[i][j])/frequency_collection_30[i][j])
            
lb_30 = np.zeros(frequency_collection_30.shape)
ub_30 = np.zeros(frequency_collection_30.shape)
for i in range(lb_30.shape[0]):
    for j in range(lb_30.shape[1]):
        if frequency_collection_30[i][j]!=0:
            lb_30[i][j] = choice_collection_30[i][j] * (1-target_z_score*stardard_error_30[i][j]) 
            ub_30[i][j] = choice_collection_30[i][j] * (1+target_z_score*stardard_error_30[i][j])
            
stardard_error_40 = np.zeros(frequency_collection_40.shape)
for i in range(stardard_error_40.shape[0]):
    for j in range(stardard_error_40.shape[1]):
        if frequency_collection_40[i][j]!=0:
            stardard_error_40[i][j] = np.sqrt((1-choice_collection_40[i][j])/frequency_collection_40[i][j])
            
lb_40 = np.zeros(frequency_collection_40.shape)
ub_40 = np.zeros(frequency_collection_40.shape)
for i in range(lb_40.shape[0]):
    for j in range(lb_40.shape[1]):
        if frequency_collection_40[i][j]!=0:
            lb_40[i][j] = choice_collection_40[i][j] * (1-target_z_score*stardard_error_40[i][j]) 
            ub_40[i][j] = choice_collection_40[i][j] * (1+target_z_score*stardard_error_40[i][j])

stardard_error_50 = np.zeros(frequency_collection_50.shape)
for i in range(stardard_error_50.shape[0]):
    for j in range(stardard_error_50.shape[1]):
        if frequency_collection_50[i][j]!=0:
            stardard_error_50[i][j] = np.sqrt((1-choice_collection_50[i][j])/frequency_collection_50[i][j])
            
lb_50 = np.zeros(frequency_collection_50.shape)
ub_50 = np.zeros(frequency_collection_50.shape)
for i in range(lb_50.shape[0]):
    for j in range(lb_50.shape[1]):
        if frequency_collection_50[i][j]!=0:
            lb_50[i][j] = choice_collection_50[i][j] * (1-target_z_score*stardard_error_50[i][j]) 
            ub_50[i][j] = choice_collection_50[i][j] * (1+target_z_score*stardard_error_50[i][j])
            
stardard_error_60 = np.zeros(frequency_collection_60.shape)
for i in range(stardard_error_60.shape[0]):
    for j in range(stardard_error_60.shape[1]):
        if frequency_collection_60[i][j]!=0:
            stardard_error_60[i][j] = np.sqrt((1-choice_collection_60[i][j])/frequency_collection_60[i][j])
            
lb_60 = np.zeros(frequency_collection_60.shape)
ub_60 = np.zeros(frequency_collection_60.shape)
for i in range(lb_60.shape[0]):
    for j in range(lb_60.shape[1]):
        if frequency_collection_60[i][j]!=0:
            lb_60[i][j] = choice_collection_60[i][j] * (1-target_z_score*stardard_error_60[i][j]) 
            ub_60[i][j] = choice_collection_60[i][j] * (1+target_z_score*stardard_error_60[i][j])            


### full data with different offertimes

In [13]:
pred_offer_times_list = [20,30,40,50,60]
pred_full_instance  = [choice_collection_20,choice_collection_30,choice_collection_40,choice_collection_50,choice_collection_60]
pred_full_lb = [lb_20,lb_30,lb_40,lb_50,lb_60]
pred_full_ub = [ub_20,ub_30,ub_40,ub_50,ub_60]

pred_full_collection = [collection_20,collection_30,collection_40,collection_50,collection_60]
pred_full_collection_size = [len(offer_times_20),len(offer_times_30),len(offer_times_40),len(offer_times_50),len(offer_times_60)]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
#pred_instance_size = [100,100,100,50,50]
pred_instance_size = [50,50,50,50,50]

price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

pred_full_offertimes = [offer_times_20,offer_times_30,offer_times_40,offer_times_50,offer_times_60]

train-test split

In [14]:
all_train_instances = []
all_train_collection = []
all_test_instance = []
all_test_collection = []
all_train_lb = []
all_train_ub = []
all_train_offertimes = []

for i in range(len(pred_instance_size)):
    train_instances_collection = []
    train_collection = []
    train_lb_collection = []
    train_ub_collection = []
    train_offertimes_collection = []
    
    test_instance_collection = []
    test_collection = []
    
    test_index_collection = []
    
    
    for j in range(pred_instance_size[i]):
        
        curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
        curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
        curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
        
        curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
        
        curr_train_collection = []
        curr_test_collection = []
        curr_offertimes_lst = []
        
        while(len(curr_train_collection)==0):
            
            chosen_test_idx = random.sample(range(len(pred_full_collection[i])),pred_test_collection_size[i])
            chosen_test_idx.sort()
            
            chosen_train_idx = [x for x in range(len(pred_full_collection[i])) if x not in chosen_test_idx ]
    
            for k in range(len(chosen_train_idx)):
                curr_train_collection.append(pred_full_collection[i][chosen_train_idx[k]])
                curr_train_instance[:,k] = pred_full_instance[i][:,chosen_train_idx[k]]
                curr_train_lb[:,k] = pred_full_lb[i][:,chosen_train_idx[k]]
                curr_train_ub[:,k] = pred_full_ub[i][:,chosen_train_idx[k]]
                
                curr_offertimes_lst.append(pred_full_offertimes[i][chosen_train_idx[k]])
                
            for k in range(len(chosen_test_idx)):
                curr_test_collection.append(pred_full_collection[i][chosen_test_idx[k]])
                curr_test_instance[:,k] = pred_full_instance[i][:,chosen_test_idx[k]]
                
            ## check if the training instance includes all the testing products in the test instance
            # ensure each test product has been offered at least  once
            new_lst_train = sum(curr_train_collection,[])
            unique_numbers_train = list(set(new_lst_train)) 
            
            new_lst_test = sum(curr_test_collection,[])
            unique_numbers_test = list(set(new_lst_test)) 
            
            for num in unique_numbers_test:
                if num not in unique_numbers_train:
                    #print('the unique train numbers',unique_numbers_train)
                    #print('the unique test numbers',unique_numbers_test)
                    curr_train_collection = []
                    curr_test_collection = []
                    curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                    curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                    curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                    curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                    curr_offertimes_lst = []
            if chosen_test_idx in test_index_collection:
                #print('check if exisits repeated indexes ')
                curr_train_collection = []
                curr_test_collection = []
                curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                curr_offertimes_lst = []
            elif len(curr_test_collection)>0:
                test_index_collection.append(chosen_test_idx)
        #print('chosen training indexes',chosen_train_idx)
        #print('chosen testing indexes',chosen_test_idx)
        #print('curr train collection\n',curr_train_collection)
        #print('curr test collection\n',curr_test_collection)
        ''' df_train = pd.DataFrame(curr_train_instance)
        df_test = pd.DataFrame(curr_test_instance)
        df_offertimes = pd.DataFrame(curr_offertimes_lst)
        #print('curr train instance\n',df_train)
        #print('curr test instance\n',df_test)
        
        ## output the training and testing instance
        df_train.to_csv('instances/train_instances/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_test.to_csv('instances/test_instances/test_'+str(pred_offer_times_list[i])+'/test_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_offertimes.to_csv('instances/train_offertimes/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
         '''
        train_instances_collection.append(curr_train_instance)
        train_collection.append(curr_train_collection)
        train_lb_collection.append(curr_train_lb)
        train_ub_collection.append(curr_train_ub)
        train_offertimes_collection.append(curr_offertimes_lst)
                
        test_instance_collection.append(curr_test_instance)
        test_collection.append(curr_test_collection)
        
    all_train_instances.append(train_instances_collection)
    all_train_collection.append(train_collection)
    all_train_lb.append(train_lb_collection)
    all_train_ub.append(train_ub_collection)
    all_train_offertimes.append(train_offertimes_collection)
    all_test_instance.append(test_instance_collection)
    all_test_collection.append(test_collection)    
        

In [15]:
pd.DataFrame(all_train_instances[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.735746,0.705128,0.647154,0.773333,0.738497,0.741758,0.736842,0.745098,0.847826,0.846774,...,0.75,0.692308,0.715447,0.791667,0.78125,0.822222,0.739130,0.642857,0.714286,0.688805
1,0.264254,0.000000,0.000000,0.000000,0.000000,0.000000,0.070175,0.176471,0.130435,0.048387,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.294872,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.352846,0.000000,0.000000,0.000000,0.192982,0.000000,0.000000,0.000000,...,0.25,0.269231,0.211382,0.166667,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.226667,0.000000,0.000000,0.000000,0.078431,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.18750,0.022222,0.043478,0.035714,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,...,0.00,0.038462,0.000000,0.000000,0.03125,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.261503,0.000000,0.000000,0.000000,0.000000,0.104839,...,0.00,0.000000,0.073171,0.000000,0.00000,0.155556,0.000000,0.000000,0.142857,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.258242,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.217391,0.000000,0.142857,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.041667,0.00000,0.000000,0.000000,0.285714,0.000000,0.311195


In [16]:
pd.DataFrame(all_test_instance[0][0])

,0,1,2,3,4
0,0.91442,0.625,0.686567,0.748744,0.757576
1,0.00000,0.000,0.000000,0.000000,0.000000
2,0.00000,0.000,0.000000,0.000000,0.000000
3,0.00000,0.275,0.000000,0.000000,0.000000
4,0.00000,0.000,0.000000,0.000000,0.000000
5,0.08558,0.000,0.044776,0.060302,0.000000
6,0.00000,0.000,0.268657,0.000000,0.121212
7,0.00000,0.100,0.000000,0.000000,0.000000
8,0.00000,0.000,0.000000,0.190955,0.121212


In [17]:
pd.DataFrame(all_train_lb[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.698136,0.572138,0.612058,0.722495,0.711180,0.693512,0.586606,0.587908,0.711411,0.763453,...,0.573645,0.459156,0.610654,0.578135,0.593010,0.675416,0.503286,0.409610,0.517594,0.636856
1,0.226645,0.000000,0.000000,0.000000,0.000000,0.000000,-0.016976,0.038969,0.002530,-0.001249,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.161881,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.317749,0.000000,0.000000,0.000000,0.058341,0.000000,0.000000,0.000000,...,0.073645,0.045161,0.116555,-0.029283,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.175829,0.000000,0.000000,0.000000,-0.018539,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.009773,-0.034379,-0.066053,-0.054622,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.033645,0.000000,...,0.000000,-0.058685,0.000000,0.000000,-0.047977,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.234185,0.000000,0.000000,0.000000,0.000000,0.033976,...,0.000000,0.000000,0.012688,0.000000,0.000000,0.016388,0.000000,0.000000,-0.009499,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.209995,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.004146,0.000000,-0.009499,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.063400,0.000000,0.000000,0.000000,0.065807,0.000000,0.259247


In [18]:
pd.DataFrame(all_train_ub[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.773355,0.838119,0.682251,0.824171,0.765815,0.790005,0.887078,0.902288,0.984241,0.930095,...,0.926355,0.925459,0.820241,1.005198,0.969490,0.969028,0.974975,0.876104,0.910977,0.740753
1,0.301864,0.000000,0.000000,0.000000,0.000000,0.000000,0.157327,0.313972,0.258339,0.098024,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.427862,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.387942,0.000000,0.000000,0.000000,0.327624,0.000000,0.000000,0.000000,...,0.426355,0.493300,0.306209,0.362617,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.277505,0.000000,0.000000,0.000000,0.175402,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.365227,0.078823,0.153009,0.126051,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077123,0.000000,...,0.000000,0.135608,0.000000,0.000000,0.110477,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.288820,0.000000,0.000000,0.000000,0.000000,0.175701,...,0.000000,0.000000,0.133654,0.000000,0.000000,0.294723,0.000000,0.000000,0.295214,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.306488,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.438928,0.000000,0.295214,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.146733,0.000000,0.000000,0.000000,0.505622,0.000000,0.363144


In [19]:
all_test_collection[0][0]

[[0, 5], [0, 3, 7], [0, 5, 6], [0, 5, 8], [0, 6, 8]]

In [20]:

def find_product_notin_assortment(collection,n):
    notin_collection = []
    for i in range(len(collection)):
        notin_assortment = []
        for j in range(n):
            if j not in collection[i]:
                notin_assortment.append(j)
        notin_collection.append(notin_assortment)
    return notin_collection

In [21]:
def robust_mdm_prediction_w_ci(lb,ub,data, collection, unseen_assortment, price,produts_notin_collection):
    
    eps = 0.01
    l = len(unseen_assortment)
    n, m = data.shape

    prod_in_ass = [[] for _ in range(n)]
    for i, assort in enumerate(collection):
        for j in assort:
            prod_in_ass[j].append(i)

    # Create a Gurobi model
    model = gp.Model('robust_mdm_ci')
    model.setParam('OutputFlag', 0)

    # Define variables
    x = model.addVars(l, vtype=GRB.CONTINUOUS, name="x", lb=0) # choice probabilities of unseen assortment
    y = model.addVars(n, m, vtype=gp.GRB.CONTINUOUS, lb=0, name="y") # define for the in sample choice probability
    delta_plus = model.addVars(m, vtype=GRB.BINARY, name="delta_plus") # delta_{A,S}
    delta_minu = model.addVars(m, vtype=GRB.BINARY, name="delta_minu") # delta_{S,A}
    eta = model.addVar(lb=0, name="eta") # lambda_A 
    lam = model.addVars(m, lb=0, vtype=GRB.CONTINUOUS, name="lam") # lambda_S for all S in the collection
    delta = model.addVars(m, m, vtype=GRB.BINARY, name="delta") # delta_{S,T}

    # Normalization constraints of unseen assortment and bound on the lambda(A)
    model.addConstr(gp.quicksum(x[i] for i in range(l)) == 1, name="Normalization")
    model.addConstr(eta <= 1, name="Eta_Bound")
    
    # Add normalization constraints
    # Bounds for lambda
    for j in range(m):
        model.addConstr(sum(y[i, j] for i in range(n)) == 1, name=f"normalization_{j}")
        model.addConstr(lam[j] <= 1, name=f"Lambda_Bound_{j}")
    
    # in-sample MDM consistency
    for i in range(len(prod_in_ass)):
        if len(prod_in_ass[i]) > 0:
            for j in prod_in_ass[i]:
                for k in prod_in_ass[i]:
                    if j != k:
                        model.addConstr(lam[j] - lam[k] + delta[j, k] >= 0, name=f"Contr1_pro_{i}_aspair_{j}_{k}")
                        model.addConstr(lam[j] - lam[k] - 1 + delta[j, k] + eps * delta[j, k] <= 0,
                                        name=f"Contr2_pro_{i}_aspair_{j}_{k}")
                        model.addConstr(y[i, j] - y[i, k] - delta[j, k] + 1 >= 0,
                                        name=f"Contr3_pro_{i}_aspair_{j}_{k}")
                        model.addConstr(y[i, j] - y[i, k] - 1 + delta[k, j] <= 0,
                                        name=f"Contr4_pro_{i}_aspair_{i}_{j}_{k}")
                        model.addConstr(y[i, j] - y[i, k] + delta[j, k] + delta[k, j] >= 0,
                                        name=f"Contr5_pro_{i}_aspair_{j}_{k}")
                        model.addConstr(y[i, j] - y[i, k] - delta[j, k] - delta[k, j] <= 0,
                                        name=f"Contr6_pro_{i}_aspair_{j}_{k}")
    
    
    # Out-of-sample MDM consistency
    for k in range(len(unseen_assortment)):
        for i in prod_in_ass[unseen_assortment[k]]:
            model.addConstr(eta - lam[i] + delta_plus[i] >= 0, name=f"Cons1_{k}_{i}")
            model.addConstr(eta - lam[i] - 1 + delta_plus[i] + eps * delta_plus[i] <= 0, name=f"Cons2_{k}_{i}")
            
            model.addConstr(lam[i] - eta + delta_minu[i] >= 0, name=f"Cons3_{k}_{i}")
            model.addConstr(lam[i] - eta - 1 + delta_minu[i] + eps * delta_minu[i] <= 0, name=f"Cons4_{k}_{i}")
            
            model.addConstr(x[k] - y[unseen_assortment[k],i] - delta_plus[i] + 1 >= 0, name=f"Cons5_{k}_{i}")
            model.addConstr(x[k] - y[unseen_assortment[k],i]  + delta_minu[i] - 1 <= 0, name=f"Cons6_{k}_{i}")
            
            model.addConstr(x[k] - y[unseen_assortment[k],i]  + delta_plus[i] + delta_minu[i] >= 0,
                            name=f"Cons7_{k}_{i}")
            model.addConstr(x[k] - y[unseen_assortment[k],i]  - delta_plus[i] - delta_minu[i] <= 0,
                            name=f"Cons8_{k}_{i}")

    # In-sample MDM choice probability lies in the confidence interval
    for i in range(len(collection)):
        # normalization constraint for each assortment. 
        #model.addConstr(gp.quicksum(x[k, i] for k in collection[i]) == 1)
        for j in collection[i]:
            # constraints for the lower bound and upper bound values
            model.addConstr(y[j,i]>= lb[j,i],name=f"lb_product{j}_assortment_{i}")
            model.addConstr(y[j,i]<= ub[j,i],name=f"ub_product{j}_assortment_{i}")

        for k in produts_notin_collection[i]:
            # constraints for products that are not in the assortments
            model.addConstr(y[k,i] == 0,name=f"prob_product{k}_assortment_{i}")

    # Objective
    obj = gp.quicksum(price[unseen_assortment[i]] * x[i] for i in range(l))
    model.setObjective(obj, GRB.MINIMIZE)

    # Optimize the model
    model.optimize()

    # Access the optimal solution
    if model.status == gp.GRB.OPTIMAL:
        #model.write("robust_mdm_ci.sol")
        #model.write("robust_mdm_ci.lp")
        print(f"MDM robust prediction revenue of assortment {unseen_assortment} is : {model.objVal}") 
        y_values = [[y[i, j].X for j in range(m)] for i in range(n)]
        y_values = np.array(y_values) 
        return [model.objVal, [x[i].x for i in range(l)], [lam[i].x for i in range(m)], eta.x,y_values, model.Runtime]
    else:
        print(f'error occurred for MDM robust prediction revenue of assortment {unseen_assortment} .')
        print("Gurobi optimization status:",model.status)

    

In [22]:
def best_case_mdm_prediction_w_ci(lb,ub,data, collection, unseen_assortment, price,produts_notin_collection):
    
    eps = 0.01
    l = len(unseen_assortment)
    n, m = data.shape

    prod_in_ass = [[] for _ in range(n)]
    for i, assort in enumerate(collection):
        for j in assort:
            prod_in_ass[j].append(i)

    # Create a Gurobi model
    model = gp.Model('best_case_mdm_ci')
    model.setParam('OutputFlag', 0)

    # Define variables
    x = model.addVars(l, vtype=GRB.CONTINUOUS, name="x", lb=0) # choice probabilities of unseen assortment
    y = model.addVars(n, m, vtype=gp.GRB.CONTINUOUS, lb=0, name="y") # define for the in sample choice probability
    delta_plus = model.addVars(m, vtype=GRB.BINARY, name="delta_plus") # delta_{A,S}
    delta_minu = model.addVars(m, vtype=GRB.BINARY, name="delta_minu") # delta_{S,A}
    eta = model.addVar(lb=0, name="eta") # lambda_A 
    lam = model.addVars(m, lb=0, vtype=GRB.CONTINUOUS, name="lam") # lambda_S for all S in the collection
    delta = model.addVars(m, m, vtype=GRB.BINARY, name="delta") # delta_{S,T}

    # Normalization constraints of unseen assortment and bound on the lambda(A)
    model.addConstr(gp.quicksum(x[i] for i in range(l)) == 1, name="Normalization")
    model.addConstr(eta <= 1, name="Eta_Bound")
    
    # Add normalization constraints
    # Bounds for lambda
    for j in range(m):
        model.addConstr(sum(y[i, j] for i in range(n)) == 1, name=f"normalization_{j}")
        model.addConstr(lam[j] <= 1, name=f"Lambda_Bound_{j}")
    
    # in-sample MDM consistency
    for i in range(len(prod_in_ass)):
        if len(prod_in_ass[i]) > 0:
            for j in prod_in_ass[i]:
                for k in prod_in_ass[i]:
                    if j != k:
                        model.addConstr(lam[j] - lam[k] + delta[j, k] >= 0, name=f"Contr1_pro_{i}_aspair_{j}_{k}")
                        model.addConstr(lam[j] - lam[k] - 1 + delta[j, k] + eps * delta[j, k] <= 0,
                                        name=f"Contr2_pro_{i}_aspair_{j}_{k}")
                        model.addConstr(y[i, j] - y[i, k] - delta[j, k] + 1 >= 0,
                                        name=f"Contr3_pro_{i}_aspair_{j}_{k}")
                        model.addConstr(y[i, j] - y[i, k] - 1 + delta[k, j] <= 0,
                                        name=f"Contr4_pro_{i}_aspair_{i}_{j}_{k}")
                        model.addConstr(y[i, j] - y[i, k] + delta[j, k] + delta[k, j] >= 0,
                                        name=f"Contr5_pro_{i}_aspair_{j}_{k}")
                        model.addConstr(y[i, j] - y[i, k] - delta[j, k] - delta[k, j] <= 0,
                                        name=f"Contr6_pro_{i}_aspair_{j}_{k}")
    
    
    # Out-of-sample MDM consistency
    for k in range(len(unseen_assortment)):
        for i in prod_in_ass[unseen_assortment[k]]:
            model.addConstr(eta - lam[i] + delta_plus[i] >= 0, name=f"Cons1_{k}_{i}")
            model.addConstr(eta - lam[i] - 1 + delta_plus[i] + eps * delta_plus[i] <= 0, name=f"Cons2_{k}_{i}")
            
            model.addConstr(lam[i] - eta + delta_minu[i] >= 0, name=f"Cons3_{k}_{i}")
            model.addConstr(lam[i] - eta - 1 + delta_minu[i] + eps * delta_minu[i] <= 0, name=f"Cons4_{k}_{i}")
            
            model.addConstr(x[k] - y[unseen_assortment[k],i] - delta_plus[i] + 1 >= 0, name=f"Cons5_{k}_{i}")
            model.addConstr(x[k] - y[unseen_assortment[k],i]  + delta_minu[i] - 1 <= 0, name=f"Cons6_{k}_{i}")
            
            model.addConstr(x[k] - y[unseen_assortment[k],i]  + delta_plus[i] + delta_minu[i] >= 0,
                            name=f"Cons7_{k}_{i}")
            model.addConstr(x[k] - y[unseen_assortment[k],i]  - delta_plus[i] - delta_minu[i] <= 0,
                            name=f"Cons8_{k}_{i}")

    # In-sample MDM choice probability lies in the confidence interval
    for i in range(len(collection)):
        # normalization constraint for each assortment. 
        #model.addConstr(gp.quicksum(x[k, i] for k in collection[i]) == 1)
        for j in collection[i]:
            # constraints for the lower bound and upper bound values
            model.addConstr(y[j,i]>= lb[j,i],name=f"lb_product{j}_assortment_{i}")
            model.addConstr(y[j,i]<= ub[j,i],name=f"ub_product{j}_assortment_{i}")

        for k in produts_notin_collection[i]:
            # constraints for products that are not in the assortments
            model.addConstr(y[k,i] == 0,name=f"prob_product{k}_assortment_{i}")


    # Objective
    obj = gp.quicksum(price[unseen_assortment[i]] * x[i] for i in range(l))
    model.setObjective(obj, GRB.MAXIMIZE)

    # Optimize the model
    model.optimize()

    # Access the optimal solution
    if model.status == gp.GRB.OPTIMAL:
        #model.write("robust_mdm_ci.sol")
        #model.write("robust_mdm_ci.lp")
        print(f"MDM best_case prediction revenue of assortment {unseen_assortment} is : {model.objVal}") 
        y_values = [[y[i, j].X for j in range(m)] for i in range(n)]
        y_values = np.array(y_values) 
        return [model.objVal, [x[i].x for i in range(l)], [lam[i].x for i in range(m)], eta.x,y_values, model.Runtime]
    else:
        print(f'error occurred for MDM best_case prediction revenue of assortment {unseen_assortment} .')
        print("Gurobi optimization status:",model.status)

    

In [23]:
def row_assortment_data(data,collection):
    pre_data = copy.deepcopy(data)
    
    for j in range(data.shape[0]):
        for i in range(len(collection)):
            if j not in collection[i]:
                pre_data[j][i] = -1
    
    return pre_data

def find_indexes_nonnegative_descending(arr):
    row_indexes_sorted_desc_nonnegative = []
    row_indexes_positive_same_value = []

    for row in arr:
        # Find indexes with non-negative values
        nonnegative_row = row[row >= 0]  # Consider only non-negative values
        unique_elements, unique_indexes = np.unique(nonnegative_row, return_index=True)
        sorted_indexes_desc = np.nonzero(row >= 0)[0][unique_indexes][np.argsort(-unique_elements)]

        row_indexes_sorted_desc_nonnegative.append(sorted_indexes_desc)

        # Keep all indexes of elements with positive and same values
        positive_values = unique_elements[unique_elements > 0]
        positive_same_value_indexes = [np.where(row == val)[0] for val in positive_values if np.count_nonzero(row == val) > 1]
        row_indexes_positive_same_value.append(positive_same_value_indexes)

    return row_indexes_sorted_desc_nonnegative, row_indexes_positive_same_value

In [24]:
def mdm_feasibility(data,ranking,equal):
      
    ub = 100

    model = gp.Model('mdm_feasibility')
    model.setParam('OutputFlag', 0)
    eps = model.addVar(name='eps')
    lam = model.addVars(data.shape[1],name = 'lam')
    
    model.addConstr(eps<=ub ) # just give an upper bound (+ve) for eps, o.w., the problem is unbounded
    
    #for i in range(ranking.shape[0]): # from each product 
    for i in range(len(ranking)): # from each product 
        for j in range(len(ranking[i])-1): # the ranking of assortment in product i 
            model.addConstr(lam [ranking[i][j+1]] - lam[ranking[i][j]]-  eps >=0)
    
    for i in range(len(equal)):
        if len(equal[i])>0:
            for j in range(len(equal[i][0])-1):
                model.addConstr(lam[equal[i][0][j]]-lam[equal[i][0][j+1]]==0)
    
    model.setObjective(eps,GRB.MAXIMIZE)
    model.optimize()
    
     # Access the optimal solution
    lam_values = [lam[i].x for i in range(data.shape[1])]
    lam_values = np.array(lam_values)

    if (model.objVal> 0):
        #model.write("mdmfeas_out.sol")
        #model.write("mdmfeas_model.lp") 
        return [1,model.Runtime,lam_values]
    else:
        #model.write("mdminfeas_model.lp")
        #return [0, used_time]
        return [0,model.Runtime,lam_values]
    

In [25]:
def robust_prediction_lp(lambda_S_lst,lambda_A,prod_in_ass,assortment_A,limit_prob,price):
    # robust_prediction_lp(curr_lam,lambda_A,prod_in_ass,assortment_A,curr_limit_prob,price)
    
    model = gp.Model('robust_rev_lp')
    model.setParam('OutputFlag', 0)
    x = model.addVars(len(assortment_A),vtype=GRB.CONTINUOUS, lb=0, name='x')
    
    model.addConstr(gp.quicksum(x[i] for i in range(len(assortment_A))) == 1, name="Normalization")
    
    ## MDM consistency
    for i in range(len(assortment_A)):
        for j in range(len(prod_in_ass[assortment_A[i]])):
            if lambda_A >= lambda_S_lst[prod_in_ass[assortment_A[i]][j]]:
                model.addConstr(x[i]<= limit_prob[ assortment_A[i]][prod_in_ass[assortment_A[i]][j]])
            if lambda_A <= lambda_S_lst[prod_in_ass[assortment_A[i]][j]]:
                model.addConstr(x[i]>= limit_prob[ assortment_A[i]][prod_in_ass[assortment_A[i]][j]])
    
    # Objective
    obj = gp.quicksum(price[assortment_A[i]] * x[i] for i in range(len(assortment_A)))
    model.setObjective(obj, GRB.MINIMIZE)

    # Optimize the model
    model.optimize()
    
    if model.Status == GRB.OPTIMAL :
        return [model.objVal, [x[i].x for i in range(len(assortment_A))], model.Runtime]
    else:
        return [100000,[-1 for i in range(len(assortment_A))], model.Runtime]
    
    

In [26]:
def best_case_prediction_lp(lambda_S_lst,lambda_A,prod_in_ass,assortment_A,limit_prob,price):
    
    model = gp.Model('best_case_rev_lp')
    model.setParam('OutputFlag', 0)
    x = model.addVars(len(assortment_A),vtype=GRB.CONTINUOUS, lb=0, name='x')
    
    model.addConstr(gp.quicksum(x[i] for i in range(len(assortment_A))) == 1, name="Normalization")
    
    ## MDM consistency
    for i in range(len(assortment_A)):
        for j in range(len(prod_in_ass[assortment_A[i]])):
            if lambda_A >= lambda_S_lst[prod_in_ass[assortment_A[i]][j]]:
                model.addConstr(x[i]<= limit_prob[ assortment_A[i]][prod_in_ass[assortment_A[i]][j]])
            if lambda_A <= lambda_S_lst[prod_in_ass[assortment_A[i]][j]]:
                model.addConstr(x[i]>= limit_prob[ assortment_A[i]][prod_in_ass[assortment_A[i]][j]])
    
    # Objective
    obj = gp.quicksum(price[assortment_A[i]] * x[i] for i in range(len(assortment_A)))
    model.setObjective(obj, GRB.MAXIMIZE)

    # Optimize the model
    model.optimize()
    
    if model.Status == GRB.OPTIMAL :
        return [model.objVal, [x[i].x for i in range(len(assortment_A))], model.Runtime]
    else:
        return [-100000,[-1 for i in range(len(assortment_A))], model.Runtime]
    
    

In [27]:
## mdm milp for the worst case revenue
def mdm_revenue(curr_limit_prob, curr_lam, prod_in_ass,lambda_A_lst, assortment_A, price):
    
    robust_revenue_lst = []
    best_case_revenue_lst = []
    robust_pred_runtime = 0
    best_case_pred_runtime = 0
    #robust_prob_lst = []
    #best_case_prob_lst = []

    for j in range(len(lambda_A_lst)):
        lambda_A = lambda_A_lst[j]
        curr_robust_revenue,curr_robust_prob,curr_robust_runtime = robust_prediction_lp(curr_lam,lambda_A,prod_in_ass,assortment_A,curr_limit_prob,price)
        
        robust_revenue_lst.append(curr_robust_revenue)
        robust_pred_runtime = robust_pred_runtime + curr_robust_runtime
        #robust_prob_lst.append(curr_robust_prob)
        
        curr_best_case_revenue,curr_bestcase_prob,curr_best_case_runtime = best_case_prediction_lp(curr_lam,lambda_A,prod_in_ass,assortment_A,curr_limit_prob,price)
        best_case_revenue_lst.append(curr_best_case_revenue)
        best_case_pred_runtime = best_case_pred_runtime + curr_best_case_runtime
        #best_case_prob_lst.append(curr_bestcase_prob)
    
    robust_revenue_A = np.min(robust_revenue_lst)
    best_case_revenue_A = np.max(best_case_revenue_lst)
    
    ''' robust_prob_A = robust_prob_lst[np.argmin(robust_revenue_lst)]
    bestcase_prob_A = best_case_prob_lst[np.argmax(best_case_revenue_lst)] '''
    
    print(f'the robust revenue for assortment {assortment_A} is {robust_revenue_A} with runtime {robust_pred_runtime}')
    if robust_revenue_A>100:
        print('robust MDM prediction error ')
    print(f'the best-case revenue for assortment {assortment_A} is {best_case_revenue_A} with runtime {best_case_pred_runtime}\n')
    if best_case_revenue_A<0:
        print('best-case MDM prediction error ')
    
    ''' lb_sales_A = np.zeros(len(price))
    ub_sales_A = np.zeros(len(price))
    
    for k in range(len(assortment_A)):
        unit_price = np.zeros(n)
        unit_price[assortment_A[k]] = 1
        
        robust_product_prob_lst = []
        bestcase_product_prob_lst = []
        
        for j in range(len(lambda_A_lst)):
            if best_case_revenue_lst[j]>0:
                lambda_A = lambda_A_lst[j]
                curr_robust_prob = robust_prediction_lp(curr_lam,lambda_A,prod_in_ass,assortment_A,curr_limit_prob,unit_price)[0]
                
                robust_product_prob_lst.append(curr_robust_prob)
                
                curr_bestcase_prob = best_case_prediction_lp(curr_lam,lambda_A,prod_in_ass,assortment_A,curr_limit_prob,unit_price)[0]
                bestcase_product_prob_lst.append(curr_bestcase_prob)
        
        lb_sales_A[assortment_A[k]] = np.min(robust_product_prob_lst)
        ub_sales_A[assortment_A[k]] = np.max(bestcase_product_prob_lst)
        print(f'the sales lb for product {assortment_A[k]} in assortment {assortment_A} is {lb_sales_A[assortment_A[k]]}')
        print(f'the sales ub for product {assortment_A[k]} in assortment {assortment_A} is {ub_sales_A[assortment_A[k]]}\n')
     '''
    #return [robust_revenue_A,robust_pred_runtime,robust_prob_A,lb_sales_A, best_case_revenue_A,best_case_pred_runtime, bestcase_prob_A,ub_sales_A]
    return [robust_revenue_A,robust_pred_runtime, best_case_revenue_A,best_case_pred_runtime]


In [28]:

def mdm_prediction(curr_test_collection,curr_test_instance,price,curr_limit_prob, curr_lam, curr_train_collection):
    ''' true assortment ranking '''
    #true_choice_prob = [] 
    true_revenue = []
    
    ''' lb_mdm_rev_prob = []
    ub_mdm_rev_prob = []
    
    lb_mdm_sales = []
    ub_mdm_sales = [] '''
    
    lb_mdm_revenue = [] 
    ub_mdm_revenue = []
    
    lb_mdm_runtime = []
    ub_mdm_runtime = []
    
    prod_in_ass = [[] for _ in range(len(price))]
    for i, assort in enumerate(curr_train_collection):
        for j in assort:
            prod_in_ass[j].append(i)
    
    dec_lam = np.abs(np.sort(-curr_lam))
    lambda_A_lst = [np.max(curr_lam)+1]
    for i in range(len(dec_lam)-1):
        lambda_A_lst.append((dec_lam[i] + dec_lam[i+1])/2)
    lambda_A_lst.append(np.min(curr_lam)-1)
    
    for k in range(len(curr_test_collection)):
            
            assortment_A = curr_test_collection[k]
            
            # true revenue and true choice probabilities for current testing assortment
            curr_revenue = np.dot(curr_test_instance[:,k],price)
            true_revenue.append(curr_revenue)
            #true_choice_prob.append(curr_test_instance[:,k])
            
            # MDM revenue prediction
            print(f'revenue prediction for unseen assortment {assortment_A}')
            mdm_prediction_result = mdm_revenue(curr_limit_prob, curr_lam, prod_in_ass,lambda_A_lst, assortment_A, price)
            
            # return [robust_revenue_A,robust_pred_runtime, best_case_revenue_A,best_case_pred_runtime]
            
            ## record the revenue result
            lb_mdm_revenue.append(mdm_prediction_result[0])
            ub_mdm_revenue.append(mdm_prediction_result[2])
            
            ## record the choice probability result
            ''' lb_prob = np.zeros(n)
            lb_prob_result = mdm_prediction_result[2]
            
            ub_prob = np.zeros(n)
            ub_prob_result = mdm_prediction_result[6]
            
            for l in range(len(assortment_A)):
                lb_prob[assortment_A[l]] = lb_prob_result[l]
                ub_prob[assortment_A[l]] = ub_prob_result[l]
            
            lb_mdm_rev_prob.append(lb_prob)
            ub_mdm_rev_prob.append(ub_prob) '''
            
            ## record the runtime result
            lb_mdm_runtime.append(mdm_prediction_result[1])
            ub_mdm_runtime.append(mdm_prediction_result[3])
            
            
            ## MDM choice probability interval prediction
            ''' lb_sales_frac = mdm_prediction_result[3]
            ub_sales_frac = mdm_prediction_result[-1] '''
            
            ''' lb_mdm_sales.append(lb_sales_frac)
            ub_mdm_sales.append(ub_sales_frac) '''
    return true_revenue,lb_mdm_revenue,ub_mdm_revenue,lb_mdm_runtime,ub_mdm_runtime       
    #return true_choice_prob,true_revenue,lb_mdm_rev_prob,ub_mdm_rev_prob,lb_mdm_sales,ub_mdm_sales,lb_mdm_revenue,ub_mdm_revenue,lb_mdm_runtime,ub_mdm_runtime

In [29]:
def kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    num = len(values1)
    assert len(values2) == num, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(num), np.arange(num))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered/2 

In [30]:
# since the underlying choice probabilities are from MEM, 
# we don't need to do representability check and limit computation

## record the kendall tau distance between the predicted ranking of MDM and the truth
all_mdm_lb_distance_list = []  
all_mdm_ub_distance_list = []

## record the true revenue of the predicted best assortment
all_best_true_revenue_list = []
all_best_revenue_mdm_lb = []  
all_best_revenue_mdm_ub =[]

## record the difference in best assortment
all_mdm_lb_best_rev_diff = []
all_mdm_ub_best_rev_diff = []

## record all the true and predicted revenues
all_true_revenue_list = []
all_mdm_lb_revenue_list = []
all_mdm_ub_revenue_list = []

## choice probability under the revenue prediction 
''' all_true_choice_prob_list = [] 
all_mdm_lb_rev_prob_list = []
all_mdm_ub_rev_prob_list = []

all_mdm_lb_sales_list = []
all_mdm_ub_sales_list = [] '''

all_true_ranking = []
all_mdm_lb_ranking = []
all_mdm_ub_ranking = []

## record the runtime of MDM prediction
all_mdm_rev_lb_runtime = []
all_mdm_rev_ub_runtime = []

for i in range(len(pred_full_collection)):
    
    # define as a container for quantity of the same collection size 
    # The following lists are of the same size as the instance size
    
    true_best_rev_collection = [] # a container for groundtruth best reveue 
    mdm_lb_best_rev_collection = [] # a container for the best revenue predicted by robust MDM
    mdm_ub_best_rev_collection = [] # a container for the best revenue predicted by best-case MDM
    
    
    true_ranking_collection = [] # a container for groundtruth reveue ranking 
    mdm_lb_ranking_collection = [] # a container for ranking by robust mdm
    mdm_ub_ranking_collection = [] # a container for ranking by best-case mdm
    
    mdm_lb_distance_collection = [] # a container for kendall tau distance between true ranking and the ranking by robust mdm
    mdm_ub_distance_collection = [] # kendall tau distance between true ranking and the ranking by best-case mdm
    
    ''' true_choice_prob_collection = []
    mdm_lb_rev_prob_collection = []
    mdm_ub_rev_prob_collection = []
    mdm_lb_sales_frac_collection = []
    mdm_ub_sales_frac_collection = [] '''
    
    true_revenue_collection = []
    mdm_lb_revenue_collection = []
    mdm_ub_revenue_collection = []
    
    mdm_lb_runtime_collection = []
    mdm_ub_runtime_collection = []
    
    for j in range(pred_instance_size[i]):

        # the following lists are of the same size as number of testing assortments
        print(f'testing for offertimes {pred_offer_times_list[i]}, {j}th instance')
        ''' true assortment ranking '''
        true_choice_prob = [] 
        true_revenue = []
        
        ''' lb_mdm_rev_prob = []
        ub_mdm_rev_prob = []
        
        lb_mdm_sales = []
        ub_mdm_sales = [] '''
        
        lb_mdm_revenue = [] 
        ub_mdm_revenue = []
        
        lb_mdm_runtime = []
        ub_mdm_runtime = []
        
        row_data = row_assortment_data(all_train_instances[i][j],all_train_collection[i][j])
        ## find the decreasing ranking and equal sequence of the assortment in the data
        rank, equal = find_indexes_nonnegative_descending(row_data)
        ## representability check of MDM
        curr_rep_result = mdm_feasibility(all_train_instances[i][j],rank,equal)
        # return [1/0,model.Runtime] # 1: representable 0: non-representable
        ''' mdm_rep_collection.append(curr_rep_result[0])
        mdm_rep_runtime_collection.append(curr_rep_result[1]) '''
        
                # check if representable
        if curr_rep_result[0] > 0:
            # representable and then do prediction 
            # mdm_prediction(curr_test_instance,curr_test_collection,price,curr_train_instance,curr_train_collection,curr_train_perm_collection,unseen_assortment_perm_collection) mdm_prediction(curr_test_collection,curr_test_instance,price,curr_limit_prob, curr_lam, curr_train_collection)
            true_revenue,lb_mdm_revenue,ub_mdm_revenue,lb_mdm_runtime,ub_mdm_runtime = mdm_prediction(all_test_collection[i][j],all_test_instance[i][j],price,all_train_instances[i][j],curr_rep_result[-1], all_train_collection[i][j])
        else:
            produts_notin_collection = find_product_notin_assortment(all_train_collection[i][j],n)
            for k in range(len(all_test_collection[i][j])):
                
                unseen_assortment = all_test_collection[i][j][k]
                
                # true revenue and true choice probabilities for current testing assortment
                curr_revenue = np.dot(all_test_instance[i][j][:,k],price)
                true_revenue.append(curr_revenue)
                true_choice_prob.append(all_test_instance[i][j][:,k])
                
                # MDM revenue prediction
                # robust_mdm_prediction_w_ci(lb,ub,data, collection, unseen_assortment, price,produts_notin_collection)
                curr_mdm_lb_result = robust_mdm_prediction_w_ci(all_train_lb[i][j],all_train_ub[i][j],all_train_instances[i][j],all_train_collection[i][j],unseen_assortment,price,produts_notin_collection)
                curr_mdm_ub_result = best_case_mdm_prediction_w_ci(all_train_lb[i][j],all_train_ub[i][j],all_train_instances[i][j],all_train_collection[i][j],unseen_assortment,price,produts_notin_collection)
                
                ## record the revenue result
                lb_mdm_revenue.append(curr_mdm_lb_result[0])
                ub_mdm_revenue.append(curr_mdm_ub_result[0])
                
                ## record the choice probability result
                ''' lb_prob = np.zeros(n)
                lb_prob_result = curr_mdm_lb_result[1]
                
                ub_prob = np.zeros(n)
                ub_prob_result = curr_mdm_ub_result[1]
                
                for l in range(len(unseen_assortment)):
                    lb_prob[unseen_assortment[l]] = lb_prob_result[l]
                    ub_prob[unseen_assortment[l]] = ub_prob_result[l]
                
                lb_mdm_rev_prob.append(lb_prob)
                ub_mdm_rev_prob.append(ub_prob) '''
                
                ## record the runtime result
                lb_mdm_runtime.append(curr_mdm_lb_result[-1])
                ub_mdm_runtime.append(curr_mdm_ub_result[-1])
                
                
                ## MDM choice probability interval prediction
                ''' lb_sales_frac = np.zeros(n)
                ub_sales_frac = np.zeros(n)

                for l in range(len(unseen_assortment)):
                    unit_price = np.zeros(n)
                    unit_price[unseen_assortment[l]] = 1
                    ## unit price is a vector that takes value 1 only at unseen_assortment[l] and all zeros for other n-1 elements
                    lb_sales_frac[unseen_assortment[l]] = robust_mdm_prediction_w_ci(all_train_lb[i][j],all_train_ub[i][j],all_train_instances[i][j],all_train_collection[i][j],unseen_assortment,unit_price,produts_notin_collection)[0]
                    ub_sales_frac[unseen_assortment[l]] = best_case_mdm_prediction_w_ci(all_train_lb[i][j],all_train_ub[i][j],all_train_instances[i][j],all_train_collection[i][j],unseen_assortment,unit_price,produts_notin_collection)[0]
                
                lb_mdm_sales.append(lb_sales_frac)
                ub_mdm_sales.append(ub_sales_frac) '''
                
        true_revenue_collection.append(true_revenue)
        #true_choice_prob_collection.append(true_choice_prob)
        
        mdm_lb_revenue_collection.append(lb_mdm_revenue)   
        mdm_ub_revenue_collection.append(ub_mdm_revenue)
        #mdm_lb_rev_prob_collection.append(lb_mdm_rev_prob)
        #mdm_ub_rev_prob_collection.append(ub_mdm_rev_prob)
        
        mdm_lb_runtime_collection.append(lb_mdm_runtime)
        mdm_ub_runtime_collection.append(ub_mdm_runtime)
        
        ## sales fraction record
        #mdm_lb_sales_frac_collection.append(lb_mdm_sales)
        #mdm_ub_sales_frac_collection.append(ub_mdm_sales)
        
        # true assortment ranking for test instance [i][j]
        curr_true_rank = np.argsort(-np.array(true_revenue))  
        true_ranking_collection.append(curr_true_rank)
        # true best assortment revenue for test instance [i][j]
        true_best_rev_collection.append(true_revenue[curr_true_rank[0]])
        
        # lb mdm assortment ranking for test instance [i][j] 
        curr_mdm_lb_rank  = np.argsort(-np.array(lb_mdm_revenue)) 
        mdm_lb_ranking_collection.append(curr_mdm_lb_rank)
        mdm_lb_best_rev_collection.append(true_revenue[curr_mdm_lb_rank[0]])
        
        # ub mdm assortment ranking for test instance [i][j]
        curr_mdm_ub_rank = np.argsort(-np.array(ub_mdm_revenue)) 
        mdm_ub_ranking_collection.append(curr_mdm_ub_rank)
        mdm_ub_best_rev_collection.append(true_revenue[curr_mdm_ub_rank[0]])
        
        #  MDM kendallTau Distance for current instance [i][j]
        curr_mdm_lb_dist = kendall_tau_distance(curr_true_rank, curr_mdm_lb_rank)
        mdm_lb_distance_collection.append(curr_mdm_lb_dist)
        
        curr_mdm_ub_dist = kendall_tau_distance(curr_true_rank, curr_mdm_ub_rank)
        mdm_ub_distance_collection.append(curr_mdm_ub_dist)
    
    # for each test instance [i][j] kendall tau distance is a number
    all_mdm_lb_distance_list.append(mdm_lb_distance_collection)
    all_mdm_ub_distance_list.append(mdm_ub_distance_collection)
    
    # for each test instance [i][j] best revenue is a number
    all_best_true_revenue_list.append(true_best_rev_collection) 
    all_best_revenue_mdm_lb.append(mdm_lb_best_rev_collection)
    all_best_revenue_mdm_ub.append(mdm_ub_best_rev_collection)
    
    # for each test instance [i][j] best revenue difference is a number
    all_mdm_lb_best_rev_diff.append((np.array(true_best_rev_collection) - np.array(mdm_lb_best_rev_collection))/ np.array(true_best_rev_collection))
    all_mdm_ub_best_rev_diff.append((np.array(true_best_rev_collection) - np.array(mdm_ub_best_rev_collection))/np.array(true_best_rev_collection))
    
    # for each test instance [i][j], true revenue is a list of the same size as the number of testing assortments
    all_true_revenue_list.append(true_revenue_collection)
    #all_true_choice_prob_list.append(true_choice_prob_collection)
    all_mdm_lb_revenue_list.append(mdm_lb_revenue_collection)
    all_mdm_ub_revenue_list.append(mdm_ub_revenue_collection)
    #all_mdm_lb_rev_prob_list.append(mdm_lb_rev_prob_collection)
    #all_mdm_ub_rev_prob_list.append(mdm_ub_rev_prob_collection)
    
    #all_mdm_lb_sales_list.append(mdm_lb_sales_frac_collection)
    #all_mdm_ub_sales_list.append(mdm_ub_sales_frac_collection)
    
    # for each test instance [i][j], true ranking is a list of the same size as the number of testing assortments
    all_true_ranking.append(true_ranking_collection)
    all_mdm_lb_ranking.append(mdm_lb_ranking_collection)
    all_mdm_ub_ranking.append(mdm_ub_ranking_collection)
    
    ## for each test instance [i][j], runtime is a list of the same size as the number of testing assortments
    all_mdm_rev_lb_runtime.append(mdm_lb_runtime_collection)
    all_mdm_rev_ub_runtime.append(mdm_ub_runtime_collection)
    

testing for offertimes 20, 0th instance
Academic license - for non-commercial use only - expires 2024-12-10
Using license file /Users/autumn/gurobi.lic
MDM robust prediction revenue of assortment [0, 5] is : 0.0
MDM best_case prediction revenue of assortment [0, 5] is : 0.2777062949887603
MDM robust prediction revenue of assortment [0, 3, 7] is : 0.12423996472891838
MDM best_case prediction revenue of assortment [0, 3, 7] is : 0.2845669338686416
MDM robust prediction revenue of assortment [0, 5, 6] is : 0.11100364345591635
MDM best_case prediction revenue of assortment [0, 5, 6] is : 0.1807398453775875
MDM robust prediction revenue of assortment [0, 5, 8] is : 0.36773428594388774
MDM best_case prediction revenue of assortment [0, 5, 8] is : 0.6216435879737974
MDM robust prediction revenue of assortment [0, 6, 8] is : 0.21043815504777652
MDM best_case prediction revenue of assortment [0, 6, 8] is : 0.5805392262373301
testing for offertimes 20, 1th instance
MDM robust prediction revenue 

In [31]:
## adding details 1
for i in range(len(pred_offer_times_list)):
    df_mdm_details1 = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mdm_lb_dist':all_mdm_lb_distance_list[i],'mdm_ub_dist':all_mdm_ub_distance_list[i],
                                    'true_best_rev':all_best_true_revenue_list[i],'mdm_lb_best_rev':all_best_revenue_mdm_lb[i],'mdm_ub_best_rev':all_best_revenue_mdm_ub[i],
                                    'mdm_lb_best_rev_diff':all_mdm_lb_best_rev_diff[i],'mdm_ub_best_rev_diff':all_mdm_ub_best_rev_diff[i]})
    df_mdm_details1.to_csv('prediction/mdm/revenue_prediction/details1/'+str(pred_offer_times_list[i])+'.csv')

In [32]:
## adding details 2 about ranking, revenue, choice probability
for i in range(len(pred_offer_times_list)):
    for j in range(pred_instance_size[i]):
        
        true_rank = all_true_ranking[i][j]
        lb_rank = all_mdm_lb_ranking[i][j]
        ub_rank = all_mdm_ub_ranking[i][j]
        df_rank = pd.DataFrame({'true_rank':true_rank,'lb_rank':lb_rank,'ub_rank':ub_rank})
        df_rank.to_csv('prediction/mdm/revenue_prediction/details2/ranking/offertimes'+str(pred_offer_times_list[i])+'/rank_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        true_rev = all_true_revenue_list[i][j]
        lb_rev = all_mdm_lb_revenue_list[i][j]
        ub_rev = all_mdm_ub_revenue_list[i][j]
        df_rev = pd.DataFrame({'true_rev':true_rev,'lb_rev':lb_rev,'ub_rev':ub_rev})
        df_rev.to_csv('prediction/mdm/revenue_prediction/details2/revenue/offertimes'+str(pred_offer_times_list[i])+'/rev_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        ''' true_prob = all_true_choice_prob_list[i][j]
        lb_prob = all_mdm_lb_rev_prob_list[i][j]
        ub_prob = all_mdm_ub_rev_prob_list[i][j]
        
        df_true_prob = pd.DataFrame(true_prob).T
        df_lb_prob = pd.DataFrame(lb_prob).T
        df_ub_prob = pd.DataFrame(ub_prob).T
        
        df_true_prob.to_csv('prediction/mdm/revenue_prediction/details2/probabilities/offertimes'+str(pred_offer_times_list[i])+'/true/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_lb_prob.to_csv('prediction/mdm/revenue_prediction/details2/probabilities/offertimes'+str(pred_offer_times_list[i])+'/mdm_lb/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_ub_prob.to_csv('prediction/mdm/revenue_prediction/details2/probabilities/offertimes'+str(pred_offer_times_list[i])+'/mdm_ub/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
         '''
        lb_runtime = all_mdm_rev_lb_runtime[i][j]
        ub_runtime = all_mdm_rev_ub_runtime[i][j]
        df_runtime = pd.DataFrame({'lb_runtime':lb_runtime,'ub_runtime':ub_runtime})
        df_runtime.to_csv('prediction/mdm/revenue_prediction/details2/runtime/offertimes'+str(pred_offer_times_list[i])+'/runtime_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        
        ## using sales fraction prediction
        ## record the choice probability prediction interval
        ''' lb_sales = all_mdm_lb_sales_list[i][j]
        ub_sales = all_mdm_ub_sales_list[i][j]
        df_lb_sales = pd.DataFrame(lb_sales).T
        df_ub_sales = pd.DataFrame(ub_sales).T
        
        df_true_prob.to_csv('prediction/mdm/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/true/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_lb_sales.to_csv('prediction/mdm/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/mdm_lb/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_ub_sales.to_csv('prediction/mdm/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/mdm_ub/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv') '''

In [33]:
## adding details 2 about runtime 
all_lb_avg_runtime = []
all_ub_avg_runtime =[]

for i in range(len(pred_offer_times_list)):
    
    avg_lb_runtime_collection = []
    avg_ub_runtime_collection = []
    
    for j in range(pred_instance_size[i]):
        avg_lb_runtime_collection.append(np.mean(np.array(all_mdm_rev_lb_runtime[i][j])))
        avg_ub_runtime_collection.append(np.mean(np.array(all_mdm_rev_ub_runtime[i][j])))
    
    all_lb_avg_runtime.append(np.mean(np.array(avg_lb_runtime_collection)))
    all_ub_avg_runtime.append(np.mean(np.array(avg_ub_runtime_collection)))

avg_mdm_lb_dist = []
avg_mdm_ub_dist = []

avg_mdm_lb_best_rev_diff = []
avg_mdm_ub_best_rev_diff = []

for i in range(len(pred_offer_times_list)):
    avg_mdm_lb_dist.append(np.mean(all_mdm_lb_distance_list[i]))
    avg_mdm_ub_dist.append(np.mean(all_mdm_ub_distance_list[i]))
    
    avg_mdm_lb_best_rev_diff.append(np.mean(all_mdm_lb_best_rev_diff[i]))
    avg_mdm_ub_best_rev_diff.append(np.mean(all_mdm_ub_best_rev_diff[i]))

df_mdm_jd_summary = pd.DataFrame({'offertimes':pred_offer_times_list,'train_size':pred_train_collection_size,'test_size':pred_test_collection_size,
                                   'avg_lb_dist':avg_mdm_lb_dist,'avg_ub_dist':avg_mdm_ub_dist,
                                   'avg_lb_best_rev_diff':avg_mdm_lb_best_rev_diff,'avg_ub_best_rev_diff':avg_mdm_ub_best_rev_diff,
                                   'avg_lb_runtime':all_lb_avg_runtime,'avg_ub_runtime':all_ub_avg_runtime})
df_mdm_jd_summary.to_csv('prediction/mdm/jd_mdm_summary.csv')

In [34]:
df_mdm_jd_summary

,offertimes,train_size,test_size,avg_lb_dist,avg_ub_dist,avg_lb_best_rev_diff,avg_ub_best_rev_diff,avg_lb_runtime,avg_ub_runtime
0,20,24,5,3.22,2.64,0.114613,0.089129,0.183617,0.178308
1,30,20,4,2.44,1.70,0.172931,0.080353,0.108029,0.105358
2,40,16,3,1.24,0.82,0.157314,0.084057,0.053175,0.049550
3,50,12,3,0.96,0.82,0.164254,0.062247,0.026593,0.024457
4,60,11,2,0.32,0.36,0.080134,0.092613,0.019764,0.019065


In [ ]:
## compute stardard error information